In [36]:
import pandas as pd
from pathlib import Path
from collections import Counter

In [2]:
DATA_PATH = Path("/home/pengfei/data/pdtb_v2/data/raw/wsj/")

In [3]:
pdtb2 = pd.read_csv("../pdtb2/pdtb2.csv")

/home/pengfei/miniconda3/envs/stanfordnlp/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3049: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def _get_span_list(span):
    """
    Args:
            span(str): "34..96;97..101"
    Returns:
            ret(list[list[str]]):
            if one arg has one segment: [[34,96]]
            if one arg has two segment: example [[34, 96], [97, 101]]
    """
    if type(span) == float:
        return []
    spans = span.split(';')
    return [o.split('..') for o in spans]

In [5]:
for i in range(100):
    arg1_char_span_string = pdtb2.loc[i, 'Arg1_SpanList']
    arg1_char_span_list = _get_span_list(arg1_char_span_string)
    arg1_rawtext = pdtb2.loc[i, 'Arg1_RawText']
    section = pdtb2.loc[i, 'Section']
    filenumber = pdtb2.loc[i, 'FileNumber']
    section = '{0:02}'.format(section)
    filename = 'wsj_{0:02}'.format(int(section)) + '{:02}'.format(filenumber)
    with open(DATA_PATH/section/filename) as f:
        doc_rawtext = f.read()
#     print(doc_rawtext[int(first):int(second)] == arg1_rawtext or doc_rawtext[int(first)+1:int(second)+1] == arg1_rawtext, i)

in terms of segment number:
single, double, trible = (38584, 1948, 51)

In [7]:
def check_length(char_span_list):
    char_span = 0
    for span in char_span_list:
        char_span += int(span[1]) - int(span[0])
    return char_span

In [58]:
a = []
for i in range(40600):
    arg1_char_span_string = pdtb2.loc[i, 'Arg1_SpanList']
    arg1_char_span_list = _get_span_list(arg1_char_span_string)
    arg1_rawtext = pdtb2.loc[i, 'Arg1_RawText']

    first = arg1_char_span_list[0][0]
    second = arg1_char_span_list[0][1]
    if ';' not in arg1_char_span_string:
        char_span = check_length(arg1_char_span_list)
        a.append(abs(len(arg1_rawtext) - char_span))
    if ';' in arg1_char_span_string:
        char_span = check_length(arg1_char_span_list)
        a.append(len(arg1_rawtext) - char_span)

How many distances away:
    Counter({0: 37713,
         1: 2598,
         2: 193,
         6: 7,
         12: 1,
         9: 3,
         3: 50,
         5: 5,
         18: 2,
         4: 18,
         13: 1,
         8: 4,
         15: 1,
         62: 1,
         19: 1,
         10: 1,
         16: 1})

In [56]:
a = []
for i in range(40600):
    arg1_char_span_string = pdtb2.loc[i, 'Arg1_SpanList']
    arg1_char_span_list = _get_span_list(arg1_char_span_string)
    arg1_rawtext = pdtb2.loc[i, 'Arg1_RawText']

    
    char_span = check_length(arg1_char_span_list)
    if abs(len(arg1_rawtext) - char_span) > 2:
        a.append(len(arg1_rawtext))
    if len(arg1_rawtext) == 2283:
        print(arg1_rawtext)
        print(char_span)

With regard to Greece's long-bubbling bank-looting scandal, Mr. Papandreou's principal accuser remains George Koskotas, former owner of the Bank of Crete and self-confessed embezzler, now residing in a jail cell in Salem, Mass., from where he is fighting extradition proceedings that would return him to Greece.Mr. Koskotas's credibility is, at best, problematic.He has ample motive to shift the blame, and his testimony has also been found less than forthright on numerous points.Nevertheless, the New Democracy and Communist parties herald his assertions as proof of PASOK complicity. Among unanswered questions are whether Mr. Papandreou received $23 million of stolen Bank of Crete funds and an additional $734,000 in bribes, as contended; whether the prime minister ordered state agencies to deposit some $57 million in Mr. Koskotas's bank and then skim off the interest; and, what PASOK's cut was from the $210 million Mr. Koskotas pinched. Two former ministers were so heavily implicated in th

In [59]:
counter = Counter(a)

In [60]:
sorted(counter.keys())

[-5, -4, -3, -2, -1, 0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 12, 13, 15, 16, 18, 19, 62]

In [57]:
# write functions

In [ ]:
def correct_by_shift(char_span_list, rawtext, arg_rawtext, number):
    """
    Args:
        char_span_list: shift by number"""
    